# Дмитрий Ильин. Практическое задание с использованием Transformer.

Модель bert-base-uncased. \
Итоговый скор - 0.88888.

In [ ]:
import os
import io
import re
import time
import shutil
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#!pip install transformers[torch] -U
#!pip install accelerate -U

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
#!pip install optuna

In [ ]:
import optuna

In [ ]:
#!pip install wandb

In [ ]:
import wandb
wandb.login(key="86a209851a552b4b869e344890f0b9b209e9a482")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
root_path = '/content/drive/My Drive/MIPT/NLP/HW2/'
output_path = '/content/drive/My Drive/MIPT/NLP/HW2/output/'
logs_path = '/content/drive/My Drive/MIPT/NLP/HW2/logs/'
models_path = '/content/drive/My Drive/MIPT/NLP/HW2/models/'

In [ ]:
# !!! Удаление временных файлов !!!
for dirname, _, filenames in os.walk(output_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        #os.remove(os.path.join(dirname, filename))

# 1. Загрузка, предобработка и разбиение данных

Для трансформеров было решено делать минимальную предобработку текста, так как опытным путем было выяснено, что удаление пунтуации, стоп-слов, эмодзи, симолов не из латинского алфавита, в конечном итоге негативно влиет на финальный скор, независимо от того, применяются ли я эти меры вместе или по отдельности.\
Поэтому было оставлено только удаление ссылок, а упоминания пользователей превращено просто в символы @.

In [ ]:
def preprocess_text(text):
    text = text.lower()

    # Удаляем ссылки
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # Удаляем упоминания пользователей вида @юзер
    text = re.sub(r'@[\w_]+', '@', text)

    return text.strip()

In [ ]:
full_df = pd.read_csv(os.path.join(root_path, 'train.csv'))
full_df = full_df.dropna()
full_df['Text'] = full_df['Text'].apply(preprocess_text)
full_df.head(5)

,Unnamed: 0,Text,Sentiment
0,0,@ @ @ and and,Neutral
1,1,advice talk to your neighbours family to excha...,Positive
2,2,coronavirus australia: woolworths to give elde...,Positive
3,3,my food stock is not the only one which is emp...,Positive
4,4,"me, ready to go at supermarket during the #cov...",Extremely Negative


In [ ]:
full_df.shape[0]

41155

In [ ]:
full_df['Sentiment'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [ ]:
full_df['Text'].str.len().max()

316

In [ ]:
test_df = pd.read_csv(os.path.join(root_path, 'test.csv'))
test_df['Text'] = test_df['Text'].apply(preprocess_text)

test_df.head(5)

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending: new yorkers encounter empty supermar...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,when i couldn't find hand sanitizer at fred me...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find out how you can protect yourself and love...
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,#panic buying hits #newyork city as anxious sh...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,#toiletpaper #dunnypaper #coronavirus #coronav...


In [ ]:
test_df['Text'].str.len().max()

307

# 2. Токенизация и подготовка датасетов

На этой стадии делается токенизация только фуллсета и тестового датасета, а разбиение на валидационную и обучающую выборки будет происходить на стадии перебора гиперпараметров.

In [ ]:
label2id = {
    'Neutral': 0,
    'Positive': 1,
    'Extremely Negative': 2,
    'Negative': 3,
    'Extremely Positive': 4
}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Превращаем в списки
test_texts = test_df['Text'].tolist()
full_texts = full_df['Text'].tolist()
full_labels = [label2id[label] for label in full_df['Sentiment'].tolist()]

# Токенизируем
full_encodings = tokenizer(full_texts, truncation=True, padding=True, max_length=384)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=384)

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
full_dataset = SentimentDataset(full_encodings, full_labels)
test_dataset = SentimentDataset(test_encodings)

# 3. Подбор гиперпараметров и обучение



In [ ]:
# Определение метрики
def compute_metrics(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, pred_labels)
    return {"accuracy": acc}

In [ ]:
# Функция для очистки чекпоинтов для экономии дискового пространства
def clear_checkpoints(path, keep=2):
    try:
        checkpoints = sorted([os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))], key=lambda x: int(x.split('-')[-1]))
        for checkpoint in checkpoints[:-keep]:
            shutil.rmtree(checkpoint)
    except Exception as e:
        print("Ошибка очистки чекпоинтов:", e)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    print("Using CPU")

Using GPU: NVIDIA A100-SXM4-40GB


Здесь хотелось бы отметить, что значения per_device_train_batch_size и per_device_eval_batch_size были подобраны ранее вручную, а их автоматизированный перебор или просто изменение негативно отражалось на финальном скоре. \
Значения закомментированных гиперпараметров будут перебираться позже.

In [ ]:
# Настройка аргументов для обучения
training_args = TrainingArguments(
    output_dir=output_path,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_dir=logs_path,
    logging_steps=100,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    save_total_limit=2, # Для экономии дискового пространства
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    #num_train_epochs=3,
    #learning_rate=5e-5,
    #lr_scheduler_type="cosine",
    #warmup_ratio=0.1,
    #weight_decay=0.01
)

In [ ]:
number_of_models = 3

## 3.1. Подбор гиперпараметров

На этом шаге происходит следующее:
1. Берётся сэмпл из 25% случайных записей от фуллсета (для ускорения процесса).
2. Сэмпл разбивается на валидационную и обучающую выборки (20/80).
3. С помощью hyperparameter_search делается подбор следующих гиперпараметров: learning_rate, num_train_epochs, weight_decay, warmup_ratio, lr_scheduler_type.
4. Каждый подбор состоит из 4 попыток (на большее не хватило ресурсов), всего делается 3 подбора, а на выходе получается список из трех наборов подобранных гиперпараметров, которые сохраняются в виде json-файлов на диск.

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 4, 6),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.1, step=0.01),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.3),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "constant"]),
    }

hyperparameters = []

for i in range(number_of_models):
    print(f"Подбор гиперпараметров, итерация №{i+1}")

    print("Разбиваем датасет и токенизируем его...")

    # Разбиваем на обучающий и проверочный наборы
    sample_df = full_df.sample(frac=0.25, random_state=int(time.time()))
    train_df, val_df = train_test_split(sample_df, test_size=0.2, random_state=int(time.time()))

    # Превращаем в списки
    train_texts = train_df['Text'].tolist()
    train_labels = [label2id[label] for label in train_df['Sentiment'].tolist()]
    val_texts = val_df['Text'].tolist()
    val_labels = [label2id[label] for label in val_df['Sentiment'].tolist()]

    # Токенизация
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=384)
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=384)

    train_dataset = SentimentDataset(train_encodings, train_labels)
    val_dataset = SentimentDataset(val_encodings, val_labels)

    # Инициализация трейнера
    trainer = Trainer(
        model_init=lambda: BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    print("Подбираем гиперпараметры...")
    start_time = time.time()
    best_run = trainer.hyperparameter_search(n_trials=4, direction="maximize", hp_space=my_hp_space)
    elapsed_time = (time.time() - start_time) / 60
    print(f"Подбор гиперпараметров завершен. Время выполнения: {elapsed_time:.2f} минут")

    hyperparameters.append(best_run.hyperparameters)

    # Сохранение гиперпараметров в файл
    file_path = os.path.join(models_path, f"hyperparameters_{i}.json")
    with open(file_path, 'w') as f:
        json.dump(best_run.hyperparameters, f)

    clear_checkpoints(output_path)  # Очистка старых чекпоинтов

## 3.2. Обучение

На этом шаге происходит следующее:
1. Для обучения берется набор из трех ранее подобранных гиперпараметров.
2. На каждом их трех наборов делается обучение на фуллсете отдельной модели, после чего модель сохраняется на диск, а также добавляется в список, который будет использоваться для предсказаний в следующем шаге.

In [ ]:
hyperparameters = []

for i in range(number_of_models):
    file_path = os.path.join(models_path, f"hyperparameters_{i}.json")
    with open(file_path, 'r') as f:
        hyperparameters.append(json.load(f))

for h in hyperparameters:
  print(h)

{'learning_rate': 8.041828702260031e-05, 'num_train_epochs': 6, 'weight_decay': 0.05, 'warmup_ratio': 0.21082314885694764, 'lr_scheduler_type': 'constant'}
{'learning_rate': 0.00015366603166045323, 'num_train_epochs': 6, 'weight_decay': 0.07, 'warmup_ratio': 0.03482192914489106, 'lr_scheduler_type': 'linear'}
{'learning_rate': 6.738041162446587e-05, 'num_train_epochs': 6, 'weight_decay': 0.0, 'warmup_ratio': 0.019625367728294183, 'lr_scheduler_type': 'cosine'}


In [ ]:
models = []

training_args_final = TrainingArguments(**training_args.to_dict())
training_args_final.do_eval = False
training_args_final.load_best_model_at_end = False
training_args_final.metric_for_best_model = None
training_args_final.evaluation_strategy = "no"

for i in range(number_of_models):
    print(f"Обучение моделей на полном наборе данных, итерация №{i+1}")

    trainer = Trainer(
        model_init=lambda: BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5),
        args=training_args_final,
        train_dataset=full_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    #Выставляем гиперпараметры
    print(f"Используемые гиперпараметры: {hyperparameters[i]}")
    for key, value in hyperparameters[i].items():
        setattr(trainer.args, key, value)

    start_time = time.time()
    trainer.train()
    elapsed_time = (time.time() - start_time) / 60
    print(f"Обучение на полном наборе данных завершено. Время выполнения: {elapsed_time:.2f} минут")

    models.append(trainer.model)
    file_path = os.path.join(models_path, f"model_{i}")
    print(f"Сохранение модели в {file_path}")
    trainer.save_model(file_path)

    clear_checkpoints(output_path)  # Очистка старых чекпоинтов

Обучение моделей на полном наборе данных, итерация №1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Используемые гиперпараметры: {'learning_rate': 8.041828702260031e-05, 'num_train_epochs': 6, 'weight_decay': 0.05, 'warmup_ratio': 0.21082314885694764, 'lr_scheduler_type': 'constant'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,1.366300
200,1.014000
300,0.833000
400,0.761300
500,0.664300
600,0.669300
700,0.595800
800,0.575300
900,0.551700
1000,0.498700


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Используемые гиперпараметры: {'learning_rate': 6.738041162446587e-05, 'num_train_epochs': 6, 'weight_decay': 0.0, 'warmup_ratio': 0.019625367728294183, 'lr_scheduler_type': 'cosine'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,1.548400
200,1.196900
300,0.916900
400,0.775300
500,0.705300
600,0.662900
700,0.590400
800,0.577700
900,0.528800
1000,0.498800


Обучение на полном наборе данных завершено. Время выполнения: 39.73 минут
Ошибка визуализации: 'learning_rate'


# 4. Предсказание

На этом шаге берётся набор из трех ранее обученных моделей, после чего с помощью ensemble prediction делается предсказание на тестовом датасете, а результат сохраняется в csv-файл.

In [ ]:
models = []

for i in range(number_of_models):
    file_path = os.path.join(models_path, f"model_{i}")
    models.append(BertForSequenceClassification.from_pretrained(file_path))

In [ ]:
def ensemble_predictions(models, test_dataset):
    all_preds = []
    for model in models:
        trainer = Trainer(model=model, args=training_args_final, compute_metrics=compute_metrics)
        preds = trainer.predict(test_dataset).predictions
        all_preds.append(preds)

    avg_preds = np.mean(all_preds, axis=0)
    final_preds = np.argmax(avg_preds, axis=1)
    return final_preds

final_preds = ensemble_predictions(models, test_dataset)

In [ ]:
final_preds

array([2, 1, 4, ..., 0, 2, 4])

In [ ]:
id2label = {v: k for k, v in label2id.items()}
text_predictions = [id2label[pred] for pred in final_preds]
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'Sentiment': text_predictions
})
print(submission_df.head())

                                     id           Sentiment
0  787bc85b-20d4-46d8-84a0-562a2527f684  Extremely Negative
1  17e934cd-ba94-4d4f-9ac0-ead202abe241            Positive
2  5914534b-2b0f-4de8-bb8a-e25587697e0d  Extremely Positive
3  cdf06cfe-29ae-48ee-ac6d-be448103ba45            Negative
4  aff63979-0256-4fb9-a2d9-86a3d3ca5470             Neutral


In [ ]:
file_path = os.path.join(root_path, f"submission.csv")
submission_df.to_csv(file_path, index=False)

# 5. Выводы

Подбор гиперпараметров с последующим использованием ensemble prediction позволил поднять скор до 0.88888, что является хорошим результатом.
Интересный момент заключается в том, что перед тем как перебирать гиперпараметры и использовать ensemble, я попробовал решить задачу самым простым способом, а именно: используя ту же модель, загрузить в нее датасет без предобработки и обучиться на нем, используя по большей части дефолтные гиперпараметры, в результате чего финальный скор оказался неожиданной высоким - 0.87704.
Я предполагаю, что при наличии вычислительных ресурсов, можно было бы сделать перебор большего числа гиперпараметров на фуллсете с большим числом итераций и обучить 4-5 моделей, вместо 3, что позволило бы получить еще больший финальный скор.